In [1]:
from tqdm import tqdm
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST
from skimage.feature import hog
from skimage import exposure
import numpy as np

# Extracting HOG Features

In [2]:

train_data = FashionMNIST(root='./Data', download=True, train=True, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=0.2860, std=0.3530)]))
test_data = FashionMNIST(root='./Data', download=True, train=False, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=0.2860, std=0.3530)]))

train_data, val_data = random_split(
    train_data, [55000, 5000], generator=torch.Generator().manual_seed(10))

data = {'train': train_data,
        'val': val_data,
        'test': test_data}

In [3]:
save_path = './Data/Features/hog_features.pt'
# batch_size = 512
batch_size = 1

features = {}
for key, value in data.items():
    print(f'\nExtracting {key} Features...')

    loader = DataLoader(value, batch_size, shuffle=False, num_workers=3)

    running_features, running_labels = [], []
    for image, label in tqdm(loader):
        # Compute HOG features for each image
        image = np.squeeze(image.numpy())
        fd= hog(image, orientations=16, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
        running_features.append(fd)
        # print(np.array(running_features).shape)
        running_labels.append(label.item())
        # print(fd.shape)

    features[key] = [np.array(running_features), np.array(running_labels)]

torch.save(features, save_path)
print("\nFeature Dict Saved to, ", save_path)


Extracting train Features...


  0%|          | 0/55000 [00:00<?, ?it/s]

100%|██████████| 55000/55000 [00:43<00:00, 1262.16it/s]



Extracting val Features...


100%|██████████| 5000/5000 [00:06<00:00, 737.83it/s] 



Extracting test Features...


100%|██████████| 10000/10000 [00:10<00:00, 921.24it/s]



Feature Dict Saved to,  ./Data/Features/hog_features.pt
